In [ ]:
#source : https://www.kaggle.com/code/erikfolkesson/detailed-explanation-custom-tokenizer/notebook

import sys
import gc

import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import roc_auc_score

from sklearn.feature_extraction.text import TfidfVectorizer

#데이터 전처리 과정에서, 토큰화 커스텀을 위해 라이브러리를 불러왔다.
from transformers import PreTrainedTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm

In [ ]:
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
sub = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')
org_train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')

#기존 제공 학습데이터 수가 너무 적어 추가로 불러왔다.
train = pd.read_csv("/kaggle/input/daigt-v4-train-dataset/train_drcat_04.csv", sep=',')
train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)

#새로운 학습데이터의 경우 라벨에 인공지능이 만들었는지를 분류해놔서 이름을 바꿔주었다.
train.rename(columns = {"label":"generated"}, inplace=True)
train.head(2)

In [ ]:
import string

#단어모음집(vocab)을 만들기 위해서 학습데이터로부터 단어를 인식하고 분류하는 작업을 거친다.
unique_words = set()
for text in train['text']:
    words = text.lower().split()
    unique_words.update(words)

unique_words = {word.strip(string.punctuation) for word in unique_words}

total_unique_words = len(unique_words)
print("Total unique words:", total_unique_words)

In [ ]:
#소문자로 변환하지 않고, 어휘의 크기도 설정했다.
LOWERCASE = False
VOCAB_SIZE = total_unique_words // 2

#BPE모델을 이용해서 토크나이저를 초기화하고, 토큰화 중 알 수 없는 단어 처리를 위해 UNK 토큰을 활용했다.
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

#토크나이저 정규화를 위한 코드고, 일관된 문자 표현을 위해서 NFC 정규화를 사용했다.
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

#토크나이저 정규화 위해서 토큰들을 정리하고 학습에 사용한다.
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

#허깅페이스에서 데이터셋을 사용한다.
dataset = Dataset.from_pandas(test[['text']])

#데이터셋 전처리를 위한 함수다.
def train_corp_iter(): 
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

#앞에서 설정한 대로 토크나이저를 학습한다.
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)

#허깅페이스의 PreTrainedTokenizerFast를 사용해서 학습된 토크나이저에 추가기능을 넣는다.
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    # pad_token="[PAD]",
    # cls_token="[CLS]",
    sep_token="[SEP]",
    # mask_token="[MASK]",
)

In [ ]:
#테스트 데이터를 토큰화 한다.
tokenized_texts_test = []
for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

#학습 데이터를 토큰화 한다.
tokenized_texts_train = []
for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

In [ ]:
def dummy(text):
    #이미 토큰화 되었으므로 그대로 출력한다.
    return text

In [ ]:
#TfidfVectorizer를 이용해서 초기화 하는 작업이다. 파라미터들은 초기화 조건들을 설정한 것이다.
vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, analyzer='word',
                             tokenizer=dummy, preprocessor=dummy, token_pattern=None, strip_accents='unicode')

#단어모음집을 만들기 위해 바로 위에서 정의한 TF-IDF 알고리즘으로 단어 모음집을 만드는 과정이다.
vectorizer.fit(tokenized_texts_test)
vocab = vectorizer.vocabulary_

#학습세트를 테스트세트의 단어모음집을 활용해서 초기화한다.
vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, vocabulary=vocab,
                             analyzer='word', tokenizer=dummy, preprocessor=dummy, token_pattern=None, 
                             strip_accents='unicode')

#학습데이터와 테스트 데이터를 TF-IDF 벡터로 변환한다.
tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)

#메모리를 해제해준다.
del vectorizer
gc.collect()

In [ ]:
y_train = train['generated'].values

In [ ]:
# 파인튜닝 과정이다.

if len(test.text.values) <= 5:
    sub.to_csv('submission.csv', index=False)
else:
    clf = MultinomialNB(alpha=0.02)
    #모델은 Adam을 사용하였다.
    model = LogisticRegression(solver='adam', max_iter=8000, tol=1e-4, penalty='l2', loss='modified_huber')
    #주요 파라미터를 정의하였다.
    p6 = {'n_iter': 1500, 'verbose': -1, 'objective': 'binary', 'metric': 'auc',
          'learning_rate': 0.05073909898961407, 'colsample_bytree': 0.726023996436955,
          'colsample_bynode': 0.5803681307354022, 'lambda_l1': 8.562963348932286,
          'lambda_l2': 4.893256185259296, 'min_data_in_leaf': 115, 'max_depth': 23, 'max_bin': 898}
    lgb = LGBMClassifier(**p6) #부스팅기법으로 LGBM과 Catboost가 활용되었다.
    cat = CatBoostClassifier(iterations=1500,
                             verbose=0,
                             l2_leaf_reg=10,
                             learning_rate=0.001,
                             allow_const_label=True, loss_function='CrossEntropy')
    weights = [0.07, 0.31, 0.31, 0.31]

    ensemble = VotingClassifier(estimators=[('mnb', clf),
                                            ('adam', model),
                                            ('lgb', lgb),
                                            ('cat', cat)
                                            ],
                                weights=weights, voting='soft', n_jobs=-1)
    ensemble.fit(tf_train, y_train)
    gc.collect()
    final_preds = ensemble.predict_proba(tf_test)[:, 1]
    sub['generated'] = final_preds
    sub.to_csv('submission.csv', index=False)
    sub